This code is part of Master's Thesis titled *Investigating the Dune Systems of Coastal Bangladesh Using Unmanned Aerial Vehicle (UAV): A Case Study at Sonadia Island.*

**Author**: [Padmanabha Chowdhury](mailto:chowpadmanabha@gmail.com), MS Student (Physical Geography), Department of Geography and Environment, University of Dhaka, Bangladesh

**Supervisor**: Dr. M. Shahidul Islam, Professor, Department of Geography and Environment, University of Dhaka, Bangladesh

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Import Packages**

In [ ]:
# Basic Data Handling Packages
import pandas as pd
import numpy as np
import random

# Package for Data Visualization
import matplotlib.pyplot as plt

# Packages for Statistical Tests
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Packages for Machine Learning and Sensitivity Analysis
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

## **Define Functions**

In [ ]:
# CALCULATE p-values of ANOVA

'''
Define Function Parameters

Function ==> ANOVA (df, group_col, numeric_cols)

df = dataframe which contains the dataset
group_col = column with the values of categorical variable
numeric_cols = columns with the independent variables

'''

def ANOVA(df, group_col, numeric_cols):
    # Store results in a list
    results = []

    # Perform ANOVA using statsmodels
    for col in numeric_cols:
        formula = f'Q("{col}") ~ C(Q("{group_col}"))'
        model = ols(formula, data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        # Extract the row corresponding to the group column
        row = anova_table.loc[f'C(Q("{group_col}"))']

        results.append({
            'col': col,
            'df': row['df'],
            'sum_sq': row['sum_sq'],
            'mean_sq': row['sum_sq'] / row['df'],
            'F': row['F'],
            'p': row['PR(>F)']
        })

    return pd.DataFrame(results)

##**Data Processing**

In this step, clean the data to check missing, null or inconsistent values. Since all factors don't have same data range, we need to standardize them. It will enhance classification performance.

In [ ]:
# Load Dataset
folder = 'drive/My Drive/Dataset'
dune=pd.read_csv(folder+'/Data.csv')
dune

In [ ]:
# Check Null Values
# print(dune.isnull().sum())

# Delete Rows with Null Values
df_dune_cleaned = dune.dropna(how='any')
print(df_dune_cleaned.isnull().sum())

In [ ]:
# STANDARDIZE VARIABLES
scaler = StandardScaler()

# Identify the Factors to Scale
cols_to_scale = df_dune_cleaned.loc[:,'Length':].columns

# Set the columns to Float before Scaling
df_dune_cleaned[cols_to_scale] = df_dune_cleaned[cols_to_scale].astype(float)
df_dune_cleaned.loc[:, cols_to_scale] = scaler.fit_transform(df_dune_cleaned.loc[:, cols_to_scale])
# df_dune_cleaned

##**Significance Analysis**

Conduct a one-way ANOVA test to check whether dune classes have significant variations among all morphological, oceanographic, meteorological and anthropogenic factors.

In [ ]:
# Calculate p-value of one-way ANOVA for each dune class
# ANOVA (df, group_col, numeric_cols)
# df = dataframe which contains the dataset
# group_col = column with the values of categorical variable
# numeric_cols = columns with the independent variables

p_value= ANOVA(df_dune_cleaned,df_dune_cleaned.columns[3],df_dune_cleaned.columns[4:26])
p_value
# print(p_value[p_value['p'] < 0.01])

##**Sensitivity Analysis**

Conduct a sensitivity analysis using the Jackknife test and XGBoost model on previously used factors of dune dynamics to identify which variables have higher relative contributions in distinguishing dune classes.

In [ ]:
# XGBoost and Jackknife Test for Sensitivity Analysis

# Define Data for Training and Testing
X = df_dune_cleaned.iloc[:, 4:26] # Independent Variables or Factors
y = df_dune_cleaned['Class_General'] # Dune Classes

# Encode Class Labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split Data for Training and Testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

# Binarize All Class Labels
classes = np.arange(len(le.classes_))
y_test_binarized = label_binarize(y_test, classes=classes)

# Train the Full Model
model_full = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model_full.fit(X_train, y_train)
y_pred_proba_full = model_full.predict_proba(X_test)
full_auc = roc_auc_score(y_test_binarized, y_pred_proba_full, average='macro')

print(f"Full model AUC: {full_auc:.4f}")

# Store AUCs without and with Only One Factor
results = {
    'feature': [],
    'without_variable': [],
    'with_only_variable': []
}

# Conduct the Jackknife Test
for feature in X.columns:

    # Without the feature
    X_train_wo = X_train.drop(columns=[feature])
    X_test_wo = X_test.drop(columns=[feature])
    model_wo = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    model_wo.fit(X_train_wo, y_train)
    y_pred_proba_wo = model_wo.predict_proba(X_test_wo)
    auc_wo = roc_auc_score(y_test_binarized, y_pred_proba_wo, average='macro')

    # With only the feature
    X_train_only = X_train[[feature]]
    X_test_only = X_test[[feature]]
    model_only = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    model_only.fit(X_train_only, y_train)
    y_pred_proba_only = model_only.predict_proba(X_test_only)
    auc_only = roc_auc_score(y_test_binarized, y_pred_proba_only, average='macro')

    results['feature'].append(feature)
    results['without_variable'].append(auc_wo)
    results['with_only_variable'].append(auc_only)

df_results = pd.DataFrame(results).set_index('feature')

# Plot Bars of the Jackknife Test Results
features = df_results.index.tolist()
n_features = len(features)

ind = np.arange(n_features + 1)
bar_height = 0.6

fig, ax = plt.subplots(figsize=(21, 18))

ax.barh(ind[:-1], df_results['without_variable'], height=bar_height, color='steelblue', label='Without Variable')
ax.barh(ind[:-1], df_results['with_only_variable'], height=bar_height, color='lightcoral', label='With Only Variable')
ax.barh(ind[-1], full_auc, height=bar_height, color='#86bf73', label='With All Variables')

ax.set_yticks(ind)
ax.set_yticklabels(features + ['Full Model'])
ax.set_xlim(0, 1)
ax.set_xticks(np.arange(0, 1.01, 0.2))
ax.set_yticklabels(features + ['Full Model'], fontsize=24, color='k')

# X-axis label
ax.set_xlabel('AUC', fontweight='bold', fontsize=24)

# Legend Format
ax.legend(loc='center left', bbox_to_anchor=(1.01, 0.5), fontsize=24, borderaxespad=0.)

# xtick Format
ax.tick_params(axis='x', colors='k', labelsize=24)

# Export High Resolution Image to Google Drive
save_path = ''
plt.savefig(save_path, dpi=800, bbox_inches='tight',facecolor='white')

plt.show()